In [23]:
import pandas as pd
import numpy as np

In [24]:
rates_df = pd.read_csv('../data/rates_data_v3.csv')
rates_df.head(3)

,Unnamed: 0,Symbol,Timestamp,liquidityRate_avg,variableBorrowRate_avg,utilizationRate_avg,stableBorrowRate_avg
0,0,DAI,2020-11-01 16:00:00,0.000899,0.003996,0.083333,0.042083
1,1,DAI,2020-11-01 17:00:00,0.000000,0.000000,0.250000,0.046250
2,2,DAI,2020-11-01 18:00:00,0.000000,0.000000,0.250000,0.046250


In [25]:
rates_df['Timestamp']

0          2020-11-01 16:00:00
1          2020-11-01 17:00:00
2          2020-11-01 18:00:00
3          2020-11-01 19:00:00
4          2020-11-01 20:00:00
                  ...         
2141197    2024-09-24 02:00:00
2141198    2024-09-24 03:00:00
2141199    2024-09-24 04:00:00
2141200    2024-09-24 05:00:00
2141201    2024-09-24 06:00:00
Name: Timestamp, Length: 2141202, dtype: object

In [26]:
# Loop through each date, attempt conversion, and print non-convertible ones
rates_df['year'] = rates_df['Timestamp'].apply(lambda x:x[0:4])
rates_df['year'] = rates_df['year'].apply(lambda x: int(x))
rates_df['year'].value_counts(dropna=False)
# it seems there is no problem with years

year
2023    595680
2022    563581
2024    476369
2021    473347
2020     32225
Name: count, dtype: int64

In [27]:
rates_df['month'] = rates_df['Timestamp'].apply(lambda x:x[5:7])
# the problem might be here because of 00 , 01, ..
rates_df['month'] = rates_df['month'].apply(lambda x: int(x) + 1)
rates_df['month'].value_counts(dropna=False)

month
8     191197
7     188558
5     186744
3     185290
9     182858
6     181299
1     180749
10    180144
4     180108
12    177302
2     166550
11    140403
Name: count, dtype: int64

In [28]:
rates_df['day'] = rates_df['Timestamp'].apply(lambda x:x[8:10])
rates_df['day'] = rates_df['day'].apply(lambda x: int(x))
print(len(rates_df['day'].value_counts(dropna=False)))
rates_df['day'].value_counts(dropna=False)
# this also seems fine

31


day
3     70582
4     70512
5     70512
7     70498
8     70491
9     70488
10    70476
12    70476
6     70468
13    70464
11    70442
21    70402
20    70398
19    70381
22    70374
14    70371
18    70369
16    70368
15    70368
17    70361
23    70346
24    70314
26    70296
25    70288
27    70288
28    70214
2     70055
1     69570
29    65879
30    64143
31    41008
Name: count, dtype: int64

In [29]:
rates_df['hour'] = rates_df['Timestamp'].apply(lambda x:x[11:13])
rates_df['hour'] = rates_df['hour'].apply(lambda x: int(x))
print(len(rates_df['hour'].value_counts(dropna=False)))
rates_df['hour'].value_counts(dropna=False)
# this also seems fine

24


hour
0     89226
1     89226
23    89225
22    89224
2     89223
3     89222
21    89222
16    89221
20    89221
17    89219
4     89219
18    89219
19    89218
5     89217
15    89216
14    89214
10    89213
12    89212
6     89211
11    89211
7     89209
9     89209
8     89207
13    89198
Name: count, dtype: int64

In [30]:
rates_df['datetime'] = pd.to_datetime(rates_df[['year', 'month', 'day', 'hour']])
# The whole problem was because of the fact that months started from 0 instead of 1

In [33]:
# testing the fact that month 9 should not have 31 days:
rates_df[(rates_df['month'] == 9) & (rates_df['day'] == 31)]

,Unnamed: 0,Symbol,Timestamp,liquidityRate_avg,variableBorrowRate_avg,utilizationRate_avg,stableBorrowRate_avg,year,month,day,hour,datetime


In [34]:
rates_df['datetime'].isna().sum()

0

In [36]:
# replacing
rates_df['Timestamp'] = rates_df['datetime']
rates_df.drop('datetime', inplace=True, axis=1)
print(rates_df['Timestamp'].isna().sum())

0


In [38]:
# cleaning
rates_df.drop(['Unnamed: 0', 'year', 'month', 'day', 'hour'], inplace=True, axis=1)
print(rates_df.columns)
print(len(rates_df))
rates_df.head(3)

Index(['Symbol', 'Timestamp', 'liquidityRate_avg', 'variableBorrowRate_avg',
       'utilizationRate_avg', 'stableBorrowRate_avg'],
      dtype='object')
2141202


,Symbol,Timestamp,liquidityRate_avg,variableBorrowRate_avg,utilizationRate_avg,stableBorrowRate_avg
0,DAI,2020-12-01 16:00:00,0.000899,0.003996,0.083333,0.042083
1,DAI,2020-12-01 17:00:00,0.000000,0.000000,0.250000,0.046250
2,DAI,2020-12-01 18:00:00,0.000000,0.000000,0.250000,0.046250


In [41]:
# checking type
type(rates_df['Timestamp'][1000])

pandas._libs.tslibs.timestamps.Timestamp

In [42]:
rates_df.to_csv('../data/rates_data_v4.csv')

### End of Notebook